In [1]:
import os
import requests
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
import pymongo as db
from pymongo import MongoClient

executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\stefa\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache


In [2]:
bf_url = "https://www.outsideonline.com/gallery/10-most-convincing-bigfoot-sightings/"

In [3]:
# # Enter whatever URL you like
browser.visit(bf_url)

# Add the page source to the variable `content`.
content = browser.html
# Load the contents of the page, its source, into BeautifulSoup 

soup = BeautifulSoup(content,features="lxml")
# print (soup.prettify)

In [4]:
soup.prettify

<bound method Tag.prettify of <html class="no-js no-svg" lang="en-US"><head>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://www.outsideonline.com/xmlrpc.php" rel="pingback"/>
<meta content="AltUrEwPn1GcXYUaHGnHR9aaM3zNhF2jC6mjfFKtvSs" name="google-site-verification"/>
<script async="" src="data:text/javascript,var%20style%20%3D%20document.createElement%28%27style%27%29%3B%0Aif%20%28document.cookie.indexOf%28%27_rm%3Dtrue%27%29%20%3D%3D%3D%20-1%29%20%7B%0A%20%20%20%20style.innerText%20%2B%3D%20%27.js-thin-banner%20%7Bdisplay%3A%20block%20%21important%7D%27%3B%7D%20else%20%7B%0A%20%20%20%20style.innerText%20%3D%20%27.o-content-cta%20%7Bdisplay%3A%20none%20%21important%7D%27%3B%0A%7D%0Aif%20%28style.innerText%29%20document.head.appendChild%28style%29%3B%0A"></script>
<!-- OneTrust Cookies Consent Noti

In [5]:
sections = []
for element in soup.findAll(attrs = {'class': 'c-content-single-gallery__gallery-image u-spacing'}):
    images = {}
    img_sect = element.find(attrs={'class':'o-image lazy'})
#     print(img_sect)
    img_dtl = img_sect["alt"]
#     print(img_dtl)
    img_src = img_sect["data-src"]
#     print(img_src)
    
    images["blurb"] = img_dtl
    images["photo"] = img_src

    sections.append(images)

sections

[{'blurb': 'For decades, people around the world have been fascinated with the legend of bigfoot, sasquatch, yeti, or whatever you choose to call it. Sightings of a furry, upright biped and reports of beastly footprints have been reported from as far afield as the Himalayas. Although no definitive proof exists, the (often questionable) reports continue adding up. Outside plunged into the deep, dark corners of this subculture to compile the most famous—perhaps most convincing—bigfoot photos ever captured. Here’s the evidence. Is bigfoot real? You be the judge.\n\nPhoto: Arguably the most famous and influential Bigfoot footage is the 1967 film shot by Roger Patterson and Bob Gimlin in Northern California. The “Bigfoot walk” it depicts has been parodied by many, but never truly replicated. Even with this enhanced image, it’s hard to tell if it’s a person in a gorilla suit or the real deal.',
  'photo': 'https://www.outsideonline.com/wp-content/uploads/2016/07/08/bigfoot-roger-patterson-1_

In [6]:
client = MongoClient()
db=client.Project_3

In [7]:
db.images.drop() #drops collection if exists to avoid duplicating

In [8]:
sect = pd.DataFrame(sections)
sect["index"] = sect.index
sect

,blurb,photo,index
0,"For decades, people around the world have been...",https://www.outsideonline.com/wp-content/uploa...,0
1,A youth group was camping in the Marble Mounta...,https://www.outsideonline.com/wp-content/uploa...,1
2,British explorer Eric Earle Shipton snapped th...,https://www.outsideonline.com/wp-content/uploa...,2
3,"Called “The Independence Day” film, this remar...",https://www.outsideonline.com/wp-content/uploa...,3
4,"In October 2012, a group of siblings hiking in...",https://www.outsideonline.com/wp-content/uploa...,4
5,"In 2007, hunter Rick Jacobs captured some of t...",https://www.outsideonline.com/wp-content/uploa...,5
6,"Locals in Johnstown, Pennsylvania, were baffle...",https://www.outsideonline.com/wp-content/uploa...,6
7,"In 1994, former U.S. Forest patrolman Paul Fre...",https://www.outsideonline.com/wp-content/uploa...,7
8,Mississippi resident Josh Highcliff captured v...,https://www.outsideonline.com/wp-content/uploa...,8
9,A hiker was walking through the Utah Hills nea...,https://www.outsideonline.com/wp-content/uploa...,9


In [9]:
lst = list(sect)
sect[lst] = sect[lst].astype('str')

In [10]:
for x in range(len(sect)):
        results=sect.iloc[x,:]
        index=results["index"]
        result=results.to_dict()
        db.images.insert_one({"index":index,"data":result})